## Copenhagen Business School
#### Home Assignment: Programming, Algorithms and Data Structures
#### Coded by Sebastian Uedingslohmann (175867)

In [229]:
# Import the required libraries
import random
import datetime
from datetime import timedelta

In [230]:
# Set start time and end time of the simulation
start_time = datetime.datetime(2024, 12, 20, 6, 0)
end_time = datetime.datetime(2024, 12, 20, 23, 0)

In [231]:
# Define class Planes and initialize the constructor
class Planes:
  def __init__(self, registration, aircraft_type, capacity, range, turnaround_time, start_time):
    self.registration = registration
    self.aircraft_type = aircraft_type
    self.capacity = capacity
    self.range = range
    self.turnaround_time = timedelta(hours = turnaround_time)
    self.available_from = start_time
    self.current_crew = None

In [232]:
# Define class Destinations and initialize the constructor
class Destinations:
  def __init__(self, iata, city, distance, flight_time):
    self.iata = iata
    self.city = city
    self.distance = distance
    self.flight_time = timedelta(hours = flight_time)
    self.estimated_demand = random.randint(30, 100) # Demand for each route is determined randomly
    self.profit_per_head = 15 # Profitability of each passenger is the same for better comparison
    self.occupancy = 0

  # Sort the destinations by profitability using merge sort
  @staticmethod
  def merge_sort(destinations):
    """
    This function splits the destinations in lists.
    """
    if len(destinations) <= 1:
      return destinations

    mid = len(destinations) // 2
    lower = Destinations.merge_sort(destinations[:mid])
    upper = Destinations.merge_sort(destinations[mid:])

    return Destinations.merge(lower, upper)

  @staticmethod
  def merge(lower, upper):
    """
    This function merges the lists and sorts them based on the estimated_demand.
    """
    destinations_sorted = []
    while lower and upper:
      if lower[0].estimated_demand > upper[0].estimated_demand:
        destinations_sorted.append(lower.pop(0))
      else:
        destinations_sorted.append(upper.pop(0))
    destinations_sorted.extend(lower if lower else upper)
    return destinations_sorted

  def calculate_profit(self, plane):
    """
    This method calculates the profit earned with a served route (forth and back).
    """
    occupancy = self.estimated_demand * (1/100) * plane.capacity * 2
    return self.profit_per_head * occupancy

In [233]:
# Define class Crew and initialize the constructor
class Crew:
  def __init__(self, id, position):
    self.id = id
    self.position = position
    self.total_hours = timedelta(hours = 0)
    self.max_working_hours = timedelta(hours = 10)
    self.available_from = datetime.datetime(2024, 12, 20, 6, 0)
    self.assigned_plane = None
    self.shift_start = None
    self.shift_end = None

  def calculate_shift_end(self, crew, travel_time):
    """
    This method calculates the time when a crew member needs to stop working at the latest to not exceed the maximum working hours.
    """
    crew.shift_end = crew.shift_start + crew.max_working_hours

  def can_work(self, current_time, flight_time):
    """
    This method returns a boolean and determines if the crew can operate the leg without exceeding the maximum working time.
    """
    return current_time + flight_time <= self.shift_end

  def add_working_hours(self, hours):
    """
    This method counts the hours the crew worked over the day.
    """
    self.total_hours += hours

In [234]:
# Define class Captain
class Captain(Crew):
  def __init__(self, id):
    super().__init__(id, position = 'Captain') # Inherits from parent class Crew

# Define class First Officer
class FirstOfficer(Crew):
  def __init__(self, id):
    super().__init__(id, position = 'First Officer') # Inherits from parent class Crew

# Define class FlightAttendant
class FlightAttendant(Crew):
  def __init__(self, id):
    super().__init__(id, position = 'Flight Attendant') # Inherits from parent class Crew

In [235]:
# Define class Simulation and initialize the constructor.
class Simulation:
  def __init__(self, planes, destinations, start_time, end_time, crew_members):
    self.planes = planes
    self.destinations = Destinations.merge_sort(destinations)
    self.start_time = start_time
    self.current_time = start_time
    self.end_time = end_time
    self.total_profit = 0
    self.no_more_flights = False
    self.crew_members = crew_members
    self.available_crew = [crew for crew in self.crew_members if crew.total_hours == timedelta(hours = 0)]

  def simulation_run(self):
    """
    This method initializes the loop to iterate over the whole day and starts the simulation.
    """
    print(f'The simulation now generates an optimal flight plan on {self.start_time.date()}. It maximizes profit and considers the maximum working hours of the employees.')

    while self.current_time < self.end_time: # The iteration goes on until the current time reaches the end of the day
      available_planes = self.available_planes()
      if self.no_more_flights: # The simulation ends earlier if the evening arrived and no destination can be served anymore, so that the plane could return to Frankfurt in time
        print(f'No more destinations can be served on this day, since the plane would return to Frankfurt after 23:00. The simulation is done. Total expected profit on {self.start_time.date()}: {self.total_profit:.2f} EUR.')
        break
      if available_planes: # If there is an available plane, the method schedule_flight is called to operate a flight
        for plane in available_planes: # Since multiple planes of the fleet might be free, the program iterates over the list of available planes to operate all of them
          if self.destinations:
            self.schedule_flight(plane)
          else:
            print('No destination left to assign. Parking all planes for the rest of the day.')
            break
      self.current_time += timedelta(minutes = 1) # The simulation iterates over the day in steps of one minute

    while True:
      user_input = input('Are you interested in the working hours of each crew member (Yes/No)? ').lower().strip()
      if user_input in ['yes', 'no']:
        break
      print('Invalid input. Please enter wither "Yes" or "No".')
    if user_input == 'yes':
      for crew in self.crew_members:
            print(f'{crew.id} worked {crew.total_hours} hours.')
    else:
      print(f'Working hours will not be printed.')

  def available_planes(self):
    """
    This method determined which planes are available at the moment.
    """
    available_planes = []
    for plane in self.planes: # Checks every plane and determines if there is an available plane in the fleet
      if plane.available_from == self.current_time:
        available_planes.append(plane) # Appends a free plane to the list
    return available_planes

  def is_in_range(self, plane, destination):
    """
    This method checks if the requested destination can be served by the aircraft type. Returns True, if the plane can fly the distance.
    """
    return plane.range >= destination.distance # Returns boolean, True of the aircraft type can cover the distance to the destination

  def travel_time_not_too_long(self, travel_time):
    """
    This method determines whether the plane would return to Frankfurt Airport in time before the night flight ban at Frankfurt Airport.
    """
    return self.current_time + travel_time <= self.end_time # Returns boolean, True if the plane would return to Frankfurt in time before the simulation ends

  def assign_crew_to_plane(self, plane, travel_time):
    """
    This method assigns a crew to an available plane. One crew can only operate one single plane on one day.
    """
    crew_plane = []
    required_roles = ['Captain', 'First Officer', 'Flight Attendant', 'Flight Attendant', 'Flight Attendant'] # Every flight needs to be operated by one captain, one first officer and three cabin crews
    for role in required_roles: # Iterate over every role
      for crew in self.available_crew: # Iterate over every staff
        if crew.position == role:
          crew_plane.append(crew) # Appends crew member to the list if it has the required role
          crew.shift_start = self.current_time # Starts counting the hours since every crew may not work longer than 10 hours
          crew.calculate_shift_end(crew, travel_time)
          self.available_crew.remove(crew) # Crew member is assigned to the plane for the day and not available to operate another plane during this day
          crew.add_working_hours(travel_time)
          break # Break the loop if this role is staffed

    if not crew_plane: # Cover the case that no crew is left for the day
      raise ValueError(f'No crew could be assigned to {plane.registration} due to lack of crew.')

    plane.current_crew = crew_plane # Assign the assembled crew to the plane
    return True

  def schedule_flight(self, plane):
    """
    This method schedules a flight depending on available planes, most profitable destinations and available crews.
    """
    for i, destination in enumerate(self.destinations): # Iterate over the sorted destinations

      if self.is_in_range(plane, destination):
        travel_time = destination.flight_time + plane.turnaround_time + destination.flight_time # Calculate travel time: Flight time forth, turnaround duration at destination airport, and flight time back to FRA

        if self.travel_time_not_too_long(travel_time):

          plane.available_from = self.current_time + travel_time + plane.turnaround_time # Occupy plane for the travel time. Also add turnaround time because the plane needs to get dispatched in Frankfurt again upon arrival
          return_time = self.current_time + travel_time # Set return time of the plane
          profit = destination.calculate_profit(plane)
          self.total_profit += profit # Sum up the profit

          if plane.current_crew == None:
            try:
              self.assign_crew_to_plane(plane, travel_time) # Call method assign_crew_to_plane if the plane does not have a crew assigned
            except ValueError as e:
              print(f'{e} {plane.registration} stays grounded.')
              return

          elif not all ( # If the plane has a crew assigned, it must be checked if the crew can continue to operate the next flight or if they would exceed the maximum working hours
            crew.can_work(self.current_time, travel_time) for crew in plane.current_crew): # Necessary because plane.current_crew is defined as a list but the method can_work cannot be applied to a list
              try:
                self.assign_crew_to_plane(plane, travel_time) # If the crew would exceed the maximum working hours with the flight the plane will operate next, then the crew must be replaced
              except ValueError as e:
                print(f'{e} {plane.registration} stays grounded.')
                return # Break loop for the plane if there is no crew to operate the plane for the rest of the day
          else:
            for crew in plane.current_crew:
              crew.add_working_hours(travel_time)

          print(f'{plane.registration} departs to {destination.city} ({destination.iata}) at {self.current_time.time()} and returns to Frankfurt (FRA) at {return_time.time()}. Expected profit: {profit:.2f} EUR '
                f'Crew: {[crew.id for crew in plane.current_crew]}') # Console output
          self.destinations.pop(i) # Destination should not be served more than once per day
          return

      else:
        print(f'Destination {destination.iata} is out of range for the aircraft {plane.registration}')
    self.no_more_flights = True # It was found that no destination can be served on this day anymore

In [236]:
# Generate some instances for class Destinations
destinations = [
    Destinations("CPH", "Copenhagen", 680, 1.2), # Passes IATA-code, city name, distance from Frankfurt Airport in kilometers, and flight time in hours
    Destinations("LHR", "London Heathrow", 780, 1.3),
    Destinations("VLC", "Valencia", 1460, 2.4),
    Destinations("LIS", "Lisbon", 2140, 3.5),
    Destinations("ARN", "Stockholm Arlanda", 1300, 2.2),
    Destinations("TFS", "Tenerife South", 3280, 4.5),
    Destinations("PMO", "Palermo", 1370, 2.3),
    Destinations("LIN", "Milan Linate", 520, 1.0),
    Destinations("ZRH", "Zurich", 300, 0.8),
    Destinations("DUS", "Dusseldorf", 230, 0.7),
    Destinations("VIE", "Vienna", 600, 1.2),
    Destinations("PMI", "Palma de Mallorca", 1252, 2.0),
    Destinations("KEF", "Reykjavik", 2400, 3.5),
    Destinations("MUC", "Munich", 300, 0.8),
    Destinations("CDG", "Paris Charles de Gaulle", 480, 1.0),
    Destinations("DUB", "Dublin", 1080, 2.0),
    Destinations("GVA", "Geneva", 450, 1.0)
]

In [237]:
# Generate some instances for class Planes
planes = [
    Planes('D-AIZI', 'A320', 168, 4100, 0.75, start_time), # Passes registration, aircraft type, capacity, range in kilometers, turnaround time in hours, and simulation start time
    Planes('D-AIZG', 'A320', 168, 4100, 0.75, start_time),
    Planes('D-AIUL', 'A320', 168, 4100, 0.75, start_time)
]

In [238]:
# Generate some instances for class Captain, FirstOfficer and FlightAttendant
crew_members = [
    Captain("CPT-001"), # Passes the crew ID
    Captain("CPT-002"),
    Captain("CPT-003"),
    Captain("CPT-004"),
    Captain("CPT-005"),
    Captain("CPT-006"),

    FirstOfficer("FO-001"),
    FirstOfficer("FO-002"),
    FirstOfficer("FO-003"),
    FirstOfficer("FO-004"),
    FirstOfficer("FO-005"),
    FirstOfficer("FO-006"),

    FlightAttendant("FA-001"),
    FlightAttendant("FA-002"),
    FlightAttendant("FA-003"),
    FlightAttendant("FA-004"),
    FlightAttendant("FA-005"),
    FlightAttendant("FA-006"),
    FlightAttendant("FA-007"),
    FlightAttendant("FA-008"),
    FlightAttendant("FA-009"),
    FlightAttendant("FA-010"),
    FlightAttendant("FA-011"),
    FlightAttendant("FA-012"),
    FlightAttendant("FA-013"),
    FlightAttendant("FA-014"),
    FlightAttendant("FA-015"),
    FlightAttendant("FA-016"),
    FlightAttendant("FA-017"),
    FlightAttendant("FA-018"),
]

In [239]:
# Create an instance for class Simulation
simulation = Simulation(planes, destinations, start_time, end_time, crew_members) # Pass the list planes, destinations, simulation start and end time as well as the crew members

In [240]:
# Initiate the simulation by calling the method simulation_run
simulation.simulation_run()

The simulation now generates an optimal flight plan on 2024-12-20. It maximizes profit and considers the maximum working hours of the employees.
D-AIZI departs to Vienna (VIE) at 06:00:00 and returns to Frankfurt (FRA) at 09:09:00. Expected profit: 4888.80 EUR Crew: ['CPT-001', 'FO-001', 'FA-001', 'FA-002', 'FA-003']
D-AIZG departs to Dublin (DUB) at 06:00:00 and returns to Frankfurt (FRA) at 10:45:00. Expected profit: 4838.40 EUR Crew: ['CPT-002', 'FO-002', 'FA-004', 'FA-005', 'FA-006']
D-AIUL departs to Palermo (PMO) at 06:00:00 and returns to Frankfurt (FRA) at 11:21:00. Expected profit: 4788.00 EUR Crew: ['CPT-003', 'FO-003', 'FA-007', 'FA-008', 'FA-009']
D-AIZI departs to Paris Charles de Gaulle (CDG) at 09:54:00 and returns to Frankfurt (FRA) at 12:39:00. Expected profit: 3780.00 EUR Crew: ['CPT-001', 'FO-001', 'FA-001', 'FA-002', 'FA-003']
D-AIZG departs to Munich (MUC) at 11:30:00 and returns to Frankfurt (FRA) at 13:51:00. Expected profit: 3679.20 EUR Crew: ['CPT-002', 'FO-002